##### Model version
- Python 3.9.18
-  Pycaret 3.2.0

### AutoML Prophet Experiment

##### MLflow


In [ ]:
# import pickle
# import mlflow
# import pandas as pd
# from fbprophet import Prophet
# from pycaret.datasets import get_data

# def train_and_log_prophet(data, experiment_name='forecasting_experiment_with_Prophet'):
#     # Start a single MLflow run for logging all models
#     with mlflow.start_run():

#         # Log experiment details
#         mlflow.log_param("experiment_name", experiment_name)

#         # Loop through each time series
#         for column in data.columns:
#             # Prepare data for Prophet
#             df_prophet = data[[column]].reset_index()
#             df_prophet = df_prophet.rename(columns={'Date': 'ds', column: 'y'})

#             # Create and train the Prophet model
#             model = Prophet(yearly_seasonality=True, weekly_seasonality=True, daily_seasonality=False)
#             model.fit(df_prophet)

#             # Optional: Save the model object
#             model_filename = f"prophet_model_{column}.pkl"
#             with open(model_filename, "wb") as f:
#                 pickle.dump(model, f)

#             # Log the model with MLflow using its name
#             mlflow.log_param("column_name", column)
#             mlflow.log_metric("Prophet_MAE", model.predict(df_prophet).yhat.abs().mean())
#             mlflow.sklearn.log_model(model, f"prophet_{column}")  # Use a unique model name for logging

# # ... data preparation and setup
# data = get_data('pycaret_downloads')
# data['Date'] = pd.to_datetime(data['Date'])
# data.set_index('Date', inplace=True)

# # Train and log Prophet models for each time series
# train_and_log_prophet(data)


##### AutoML Pycaret Experiment

- 1st version

In [16]:
import os
import pickle
import mlflow
import pandas as pd
from pycaret.time_series import *
from pycaret.datasets import get_data

def train_and_log_best_model(data,fh,experiment_name='forecasting_experiment_with_PyCaret'):
    """
    Train AutoML forecasting model using PyCaret, save the best model with pickle, and log metrics using MLflow.
    
    Parameters:
    - data: DataFrame, the input dataset
    - experiment_name: str, the name of the MLflow experiment
    
    Returns:
    - None
    """

    mlflow.autolog()
    with mlflow.start_run():
        # Set up PyCaret
        s = setup(data, fh=fh, session_id=123)
        # Log experiment to MLflow
        mlflow.set_experiment("Pycaret_experiment")
        # Train models
        check_stats()
        
        best = compare_models()
        # Log PyCaret experiment
        mlflow.log_params({'fh': fh, 'session_id': 123})


# ... data preparation and setup
data = get_data('pycaret_downloads')
data['Date'] = pd.to_datetime(data['Date'])
data.set_index('Date', inplace=True)

# Train and log the best model without saving parameters
train_and_log_best_model(data, fh=36)

,Date,Total
0,7/8/2021,3773
1,7/9/2021,3172
2,7/10/2021,2079
3,7/11/2021,2725
4,7/12/2021,4542


2024/03/07 10:21:59 INFO mlflow.tracking.fluent: Autologging successfully enabled for xgboost.
2024/03/07 10:22:00 INFO mlflow.tracking.fluent: Autologging successfully enabled for sklearn.
2024/03/07 10:22:00 INFO mlflow.tracking.fluent: Autologging successfully enabled for statsmodels.
2024/03/07 10:22:00 INFO mlflow.tracking.fluent: Autologging successfully enabled for lightgbm.


MissingConfigException: Yaml file 'C:\Users\zeesh\OneDrive - ORMAE\Documents\Projects\Self_Programming_Practice\Zee_github_projects\sales_forecasting\Pycaret_forecast\mlruns\0\meta.yaml' does not exist.

In [ ]:
import mlflow
from pycaret.timeseries import *

# Set up PyCaret
s = setup(df, fh=100, session_id=123)

# Log experiment to MLflow
mlflow.set_experiment("your_experiment_name")

# Train models
check_stats()
best = compare_models()

# Log best model to MLflow
with mlflow.start_run():
    # Log PyCaret experiment
    mlflow.log_params({'fh': 100, 'session_id': 123})
    
    # Log PyCaret best model
    mlflow.log_params({'best_model': best})
    
    # Save PyCaret best model to MLflow
    save_model(best, 'best_model')

# Optional: Save the entire PyCaret setup to MLflow
save_experiment(experiment_name='your_experiment_name', verbose=False)

# Optional: Log artifacts such as feature importance plots or evaluation plots to MLflow
mlflow.log_artifacts('path_to_artifacts_folder')


## Testing accuracy of the model

In [ ]:
import mlflow
from pycaret.datasets import get_data
#...data preparation and setup
model_name='PyCaret_model'
test_set = get_data('pycaret_downloads')
test_set['Date'] = pd.to_datetime(test_set['Date'])
test_set.drop('Total', axis=1, inplace=True)

logged_model = 'runs:/344c3662bafe48389d0a0aaa834ac5ee/model'

# Load model as a PyFuncModel.
loaded_model = mlflow.pyfunc.load_model(logged_model)

# Predict on a Pandas DataFrame.
import pandas as pd
loaded_model.predict(test_set)